<a href="https://colab.research.google.com/github/NeuralClassifier/OutlierDetect/blob/main/ODRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from scipy.spatial import distance
import math

In [15]:
!git clone https://github.com/NeuralClassifier/OutlierDetect

Cloning into 'OutlierDetect'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [16]:
pd.read_csv('/content/OutlierDetect/Data/ionosphere.csv')

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,label
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,0.89391,0.13130,0.81197,0.06723,0.79307,-0.08929,1.00000,-0.02101,0.96639,0.06618,0.87605,0.01155,0.77521,0.06618,0.95378,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,0.96510,0.03281,0.94171,0.07330,0.94625,-0.01326,0.97173,0.00140,0.94834,0.06038,0.92670,0.08412,0.93124,0.10087,0.94520,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,0.94124,0.01766,0.92595,0.04688,0.93954,-0.01461,0.94837,0.02004,0.93784,0.01393,0.91406,0.07677,0.89470,0.06148,0.93988,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,0.89724,-0.03315,0.89061,-0.01436,0.90608,-0.04530,0.91381,-0.00884,0.80773,-0.12928,0.88729,0.01215,0.92155,-0.02320,0.91050,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


In [17]:
class LoadData:

  def __init__(self,datasetFileName):
    if datasetFileName == 'ionosphere.csv':
      self.df = pd.read_csv('/content/OutlierDetect/Data/ionosphere.csv')
      print(self.df.head())


  def encode(self,features):
    if features == 0:
      print('********************* Before ***********************')
      print()
      print(self.df.head())
      print()
      le = LabelEncoder()
      self.df.iloc[:,-1] = le.fit_transform(self.df.iloc[:,-1])
      print('********************* After ***********************')
      print()
      print(self.df.head())
      print()
      return self.df
    
    elif features == 1:
      st = input('All Columns? (Y/N): ')
      if st == 'Y':
        print('********************* Before ***********************')
        print()
        print(self.df.head())
        print()
        print('********************* After ***********************')
        le = LabelEncoder()
        for i in range(self.df.shape[1]):
          self.df.iloc[:,i] = le.fit_transform(self.df.iloc[:,i])
        
        print()
        print(self.df.head())
        print()
        return self.df

      elif st == 'N':
        listToencode = []

        print('********************* Enter the list of features to encode (enter # to end) ***********************')
        print()
        fts = 0
        while fts !='#':
          fts = input()
          listToencode.append(int(fts))

        print('********************* Before ***********************')
        print()
        print(self.df.head())
        print()
        print('********************* After ***********************')
        le = LabelEncoder()
        for i in listToencode:
          self.df.iloc[:,i] = le.fit_transform(self.df.iloc[:,i])
        
        print()
        print(self.df.head())
        print()
        return self.df




      else:
        print('Invalid input ... Try again!!') 
    
    else:
      print('Invalid input ... Try again!!') 

In [18]:
ld = LoadData('ionosphere.csv')

   feature1  feature2  feature3  ...  feature33  feature34  label
0         1         0   0.99539  ...    0.18641   -0.45300      g
1         1         0   1.00000  ...   -0.13738   -0.02447      b
2         1         0   1.00000  ...    0.56045   -0.38238      g
3         1         0   1.00000  ...   -0.32382    1.00000      b
4         1         0   1.00000  ...   -0.04608   -0.65697      g

[5 rows x 35 columns]


In [19]:
df = ld.encode(0)

********************* Before ***********************

   feature1  feature2  feature3  ...  feature33  feature34  label
0         1         0   0.99539  ...    0.18641   -0.45300      g
1         1         0   1.00000  ...   -0.13738   -0.02447      b
2         1         0   1.00000  ...    0.56045   -0.38238      g
3         1         0   1.00000  ...   -0.32382    1.00000      b
4         1         0   1.00000  ...   -0.04608   -0.65697      g

[5 rows x 35 columns]

********************* After ***********************

   feature1  feature2  feature3  ...  feature33  feature34  label
0         1         0   0.99539  ...    0.18641   -0.45300      1
1         1         0   1.00000  ...   -0.13738   -0.02447      0
2         1         0   1.00000  ...    0.56045   -0.38238      1
3         1         0   1.00000  ...   -0.32382    1.00000      0
4         1         0   1.00000  ...   -0.04608   -0.65697      1

[5 rows x 35 columns]



In [55]:
df.iloc[5:,:]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,label
5,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,0.03786,-0.06302,0.00000,0.00000,-0.04572,-0.15540,-0.00343,-0.10196,-0.11575,-0.05414,0.01838,0.03669,0.01519,0.00888,0.03513,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011,0
6,1,0,0.97588,-0.10602,0.94601,-0.20800,0.92806,-0.28350,0.85996,-0.27342,0.79766,-0.47929,0.78225,-0.50764,0.74628,-0.61436,0.57945,-0.68086,0.37852,-0.73641,0.36324,-0.76562,0.31898,-0.79753,0.22792,-0.81634,0.13659,-0.82510,0.04606,-0.82395,-0.04262,-0.81318,-0.13832,-0.80975,1
7,0,0,0.00000,0.00000,0.00000,0.00000,1.00000,-1.00000,0.00000,0.00000,-1.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,1.00000,-1.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,0.00000,0.00000,1.00000,1.00000,0.00000,0.00000,0
8,1,0,0.96355,-0.07198,1.00000,-0.14333,1.00000,-0.21313,1.00000,-0.36174,0.92570,-0.43569,0.94510,-0.40668,0.90392,-0.46381,0.98305,-0.35257,0.84537,-0.66020,0.75346,-0.60589,0.69637,-0.64225,0.85106,-0.65440,0.57577,-0.69712,0.25435,-0.63919,0.45114,-0.72779,0.38895,-0.73420,1
9,1,0,-0.01864,-0.08459,0.00000,0.00000,0.00000,0.00000,0.11470,-0.26810,-0.45663,-0.38172,0.00000,0.00000,-0.33656,0.38602,-0.37133,0.15018,0.63728,0.22115,0.00000,0.00000,0.00000,0.00000,-0.14803,-0.01326,0.20645,-0.02294,0.00000,0.00000,0.16595,0.24086,-0.08208,0.38065,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,0.89391,0.13130,0.81197,0.06723,0.79307,-0.08929,1.00000,-0.02101,0.96639,0.06618,0.87605,0.01155,0.77521,0.06618,0.95378,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,1
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,0.96510,0.03281,0.94171,0.07330,0.94625,-0.01326,0.97173,0.00140,0.94834,0.06038,0.92670,0.08412,0.93124,0.10087,0.94520,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,1
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,0.94124,0.01766,0.92595,0.04688,0.93954,-0.01461,0.94837,0.02004,0.93784,0.01393,0.91406,0.07677,0.89470,0.06148,0.93988,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,1
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,0.89724,-0.03315,0.89061,-0.01436,0.90608,-0.04530,0.91381,-0.00884,0.80773,-0.12928,0.88729,0.01215,0.92155,-0.02320,0.91050,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,1


In [84]:
class ODRA:
  def __init__(self,df):
    self.data_df = df
    self.data = np.array(self.data_df.iloc[:,:-1])
    self.y = np.array(self.data_df.iloc[:,-1])
    self.k = len(self.data_df)
    self.sparseMatrix = []
  
  def distance_oneD(self,i,j):
    return math.fabs(i-j)
  
  def nn_1d(self,i,j,oneD_array):
    nearest_neighbors = []
    for cnt in range(len(oneD_array)):
      nearest_neighbors.append((self.distance_oneD(oneD_array[i],oneD_array[cnt]),cnt))
    return nearest_neighbors
  
  def k_plus1_neighbors(self,i,j,oneD_array):
    nearest_neighbors = self.nn_1d(i,j,oneD_array)
    nearest_neighbors = sorted(nearest_neighbors, key=lambda x: x[0])
    return nearest_neighbors[:self.k+1]
  
  def center_nn(self,nn,oneD_array):
    sum = 0
    for i in nn:
      sum += oneD_array[i[1]]
    return float(sum/(self.k+1))
  
  def sparseness_degree(self,i,j):
    oneD_array = self.data[:,j]
    #print(oneD_array)
    kplus1_nn = self.k_plus1_neighbors(i,j,oneD_array)
    c_ij = self.center_nn(kplus1_nn,oneD_array)
    sum = 0
    for cnt in kplus1_nn:
      #print(oneD_array[cnt[1]])
      sum += (oneD_array[cnt[1]] - c_ij)**2
    return float(sum/(self.k+1))
    #return c_ij
  
  def compute_sparseness(self,i):
    sprse = []
    for m in range(self.data.shape[1]):
      sprse.append(self.sparseness_degree(i,m))
    return sprse 

  def construct_sparse_matrix(self):
    for n in range(self.data.shape[0]):
      self.sparseMatrix.append(self.compute_sparseness(n))

In [83]:
od = ODRA(df)
od.construct_sparse_matrix()